In [1]:
from korbash_lib import PowerMeter, TensionGauge, Motor, MotorSystem, Puller, Everage, Exp_everage, PlotDisplayer, Slider
import thorlabs_apt as apt
import time
import numpy as np
import pandas as pd
import datetime
import PySimpleGUI as sg
from tkinter import messagebox as mb
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot, layout
from jupyter_ui_poll import ui_events
import ipywidgets
import matplotlib
import scipy
from scipy import stats
import random
import os
output_notebook()
pl = Puller()
random.seed()

# инициализация запускать только один раз

Loading BokehJS ...

[(21, 90113195), (21, 90113196), (21, 90113197)]


In [2]:
pl.ms.StopButton() # кнопки для остоновки моторчиков в нештатной ситуации

GridspecLayout(children=(Button(description='Stop All Motors', layout=Layout(grid_area='widget001'), style=But…

In [7]:
pl.ms.MoveToStart(zapas=1) # сдвиг моторчиков на начальные позиции для растяжки
pl.ms.motorM.MoveTo(15);

In [3]:
pl.tg.SetZeroWeight(15) # калибровка тензодатчика

-1.3231953890571984

In [3]:
x0=pl.SetH_avto() # измерение высоты молибдена

SetW finish value= 50.10438593514224
podgon a= 6.864823861791799   b= 50.079889431418536   x0= 11.436773888924165
a= 6.779789720177987   b= 50.11533463301973   x0= 11.625400897544967
data saved


In [5]:
pl.SetW(50, dw=0.1, tau=1); # натяжка волокна

SetW finish value= 49.894838054912924


In [26]:
pl.ms.motorM.MoveTo(pl.ms.motorM.position_max-2) # опускание молибдена вниз перед разогревом

0

In [30]:
pl.tg.SetZeroWeight(50,5) # калибровка тензодатчика после разогрева молибдена

-882.485617561552

In [4]:
x0=20

In [8]:
while pl.ms.IsInMotion():
    pass
dp=PlotDisplayer()
dp.CreateMaket(mainParam='time',power=pl.data[['time','power']],
                    tension=pl.data[['time','tension']],
                   motors=pl.data[['time','motorR','motorL','motorM']]
                  # v_and_a=pl.data[['time','vL','vR','aL','aR']]
              )
sl=Slider()
sl.NewSl('v',min=0.1,max=9,value=3,step=0.1)
sl.NewSl('a',min=0.1,max=9,value=7,step=0.1)
sl.NewSl('dvKof', max=3,value=0,step=0.01)
sl.NewSl('dv', max=1.01,value=0,step=0.0001)
sl.NewSl('window',min=50, max=10000,value=100, step=50)
sl.NewSl('molibdenH',min=-2.6, max=5.6,value=0.3,step=0.01)
sl.NewBtn('molibden','up molibden','doun molibden')
sl.NewBtn('end','end','end2','end3','vse','bolshe ne jmi','nu pravda ne nado', 'samiy umniy da?')
sl.NewTt('x')
sl.NewTt('L')
sl.NewTt('R')
sl.NewTt('dv')

In [9]:
downPos=pl.ms.motorM.position_max-2
t0 = time.time()
t = t0
dp.Show()
sl.Display()
pl.Clear2()
with ui_events() as poll:
    while True:
        poll(10)
        t = time.time()
        pl.Read()
        x=pl.data['x'].iloc[-1]
        v=sl.Sl['v']()
        a=sl.Sl['a']()
        dv=sl.Sl['dv']()
        ww=int(sl.Sl['window']())
        sl.ChangeValueTt('x',x)
        sl.ChangeValueTt('L',pl.ms.L_x(x))
        sl.ChangeValueTt('R',pl.ms.R_x(x))
        sl.ChangeValueTt('dv',dv)

        if not pl.ms.motorM.IsInMotion():
            if not sl.BtnFl['molibden']:
                pl.ms.motorM.MoveTo(downPos)
            else: 
                pl.ms.motorM.MoveTo(x0+sl.Sl['molibdenH']())

        if not pl.ms.IsInMotion():
            if sl.BtnFl['end']==1:
                print('ending')
                sl.BtnFl['end']=2
            elif pl.ms.PulMove(v, a, dv)==-1: sl.BtnFl['end']=1 # само движение моторчиков можно убрать
            
        if sl.BtnFl['end']>1: 
            pl.ms.Stop('M')
            pl.ms.motorM.FogotMotion()
            pl.ms.Start('M')
            pl.ms.motorM.MoveTo(downPos)
            break
            
        dp.Apdate(for_all=pl.data.iloc[-ww:])
        push_notebook()    
        time.sleep(0.04)
print('ura')
pl.Save()
with ui_events() as poll:
    while sl.BtnFl['end']<3:
        poll(10)
        pl.SetW(10,dw=1,tau=0.5)
print('vse')

xMax= 38.44613234964727 L_x(xMax)= 14.700000000000001   x= 36.20250650014535   L= -14.40994779902097 xEnd= 41.48671273533955 lEnd= 14.700000000000001
ending
ura


KeyboardInterrupt: 

In [10]:
pl.ms.End() # растановка моторчиков на нули (запускать перед отключением питания)

ending
end


In [6]:
pl.Save() # сохранение данных

data saved
